In [3]:
# Author: Arthur Prigent
# Email: aprigent@ictp.it

#### DATA ####
# https://downloads.psl.noaa.gov/Datasets/
from functions import *
import warnings
warnings.filterwarnings("ignore")
from IPython.display import Image


from datetime import *

now = datetime.now()
print(now)
date_time = now.strftime("%d/%m/%Y")
print(date_time)



2024-07-15 18:05:06.329913
15/07/2024


In [13]:

if ((date_time[:2] =='01') or
    ((date_time[:2] =='15'))
    ):
    print('yes')

    path_ersstv5 = 'https://psl.noaa.gov/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc'
    (ssta_atl3_ersst,ssta_aba_ersst,
     ssta_nino34_ersst,ssta_dni_ersst,
     ssta_cni_ersst,ssta_nni_ersst,iod_ersst) = read_data_compute_anomalies_ersstv5_for_mail(path_ersstv5)
    
    ssta_atl3_norm_ersst = ssta_atl3_ersst/ssta_atl3_ersst.std(dim='time')
    ssta_aba_norm_ersst = ssta_aba_ersst/ssta_aba_ersst.std(dim='time')
    ssta_nino34_norm_ersst = ssta_nino34_ersst/ssta_nino34_ersst.std(dim='time')
    ssta_dni_norm_ersst = ssta_dni_ersst/ssta_dni_ersst.std(dim='time')
    
    last_timesteps = [str(ssta_atl3_ersst.time[-3].values)[:7],
    str(ssta_atl3_ersst.time[-2].values)[:7],
    str(ssta_atl3_ersst.time[-1].values)[:7]]
    
    import os
    mail_sender = os.environ.get("USERNAME")
    password = os.environ.get("PASSWORD")
    email = os.environ.get("EMAILS")


    import smtplib
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.image import MIMEImage
    # Email configuration
    sender_email = mail_sender
    sender_password = password
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    


    # List of recipient email addresses
    
    emails = email.split(',')
    recipient_emails = emails

    # Create the email message
    subject = "Tropical variability update"
    body = """
    Hello!

    Please find at the following link a little update on the current state of the tropics for the last three timesteps: {last_timesteps}

    Last three SST anomalies from ERSSTv5 caculated relative to the baseline 1982/01-2023/12:

    ATL3 (std SSTA {std_atl3} degC): {ssta_atl3_norm_ersst} degC
    ABA (std SSTA {std_aba} degC): {ssta_aba_norm_ersst} degC
    NINO3.4 (std SSTA {std_nino34} degC): {ssta_nino34_norm_ersst} degC
    DNI (std SSTA {std_dni} degC): {ssta_dni_norm_ersst} degC
    IOD : {iod_ersst} degC


    In () the std of the SSTA over the period from 1982/01 to the last timestep.
    Best regards,
    Arthur Prigent

    https://aprig.github.io/ATL3_ABA_SSTa/automatic_time_series_SST.html
    """

    # Generate the plot
    plt.figure(figsize=(10, 6))
    plt.plot(ssta_atl3_ersst.time[-36:], ssta_atl3_ersst[-36:], label='ATL3')
    plt.plot(ssta_aba_ersst.time[-36:], ssta_aba_ersst[-36:], label='ABA')
    plt.plot(ssta_nino34_ersst.time[-36:], ssta_nino34_ersst[-36:], label='NINO3.4')
    plt.plot(ssta_dni_ersst.time[-36:], ssta_dni_ersst[-36:], label='DNI')
    plt.xlabel('Time')
    plt.ylabel('SSTA (degC)')
    plt.title('Sea Surface Temperature Anomalies')
    plt.legend()
    plt.grid(True)
    plot_filename = 'ssta_plot.png'
    plt.savefig(plot_filename)
    plt.close()
    
    
    # Connect to the SMTP server
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()
        # Login to your Gmail account
        server.login(sender_email, sender_password)

        for recipient_email in recipient_emails:
            # Create the email message
            message = MIMEMultipart()
            message['From'] = sender_email
            message['To'] = recipient_email
            message['Subject'] = subject

            formatted_body = body.format(
                last_timesteps=last_timesteps,
                std_atl3 = np.round(ssta_atl3_ersst.std(dim='time'),2).values,
                std_aba = np.round(ssta_aba_ersst.std(dim='time'),2).values,
                std_nino34 = np.round(ssta_nino34_ersst.std(dim='time'),2).values,
                std_dni = np.round(ssta_dni_ersst.std(dim='time'),2).values,
                ssta_atl3_norm_ersst=np.round(ssta_atl3_ersst[-3:],2).values,
                ssta_aba_norm_ersst=np.round(ssta_aba_ersst[-3:],2).values,
                ssta_nino34_norm_ersst=np.round(ssta_nino34_ersst[-3:],2).values,
                ssta_dni_norm_ersst=np.round(ssta_dni_ersst[-3:],2).values,
                iod_ersst=np.round(iod_ersst[-3:],2).values

            )
            # Attach the plot image
            with open(plot_filename, 'rb') as img:
                img_data = img.read()
                image = MIMEImage(img_data, name=os.path.basename(plot_filename))
                message.attach(image)

            server.sendmail(mail_sender, recipient_email, message.as_string())

    # Quit the server
    print("Emails sent successfully!")






yes


UnicodeEncodeError: 'ascii' codec can't encode character '\xa0' in position 32: ordinal not in range(128)